# Lab Week 9: Constrained Optimization

_Tutors: Andreas Grothey, Josh Fogg_

7 March 2025

This lab session is concerned with solving nonlinear constrained problems by penalty methods and the SLP method. It highlights some concepts that are important when trying to implement these methods in practice. We make use of a couple of test problem (the first one of which is of practical importance).There is again an Assessed Task at the end (which follows on from tasks 1(c) and 3(e)).

First download the python modules required for today's session from Learn (`NLOLab5.zip`) and unpack it.

## 1. Pooling Problem by $L_2$-Penalty function

The picture below on the right gives a small example (`haverly1`) of the pooling problem: given are 3 input components (in the example crude oil grades) that have sulphur content of 3%, 1% and 2% respectively. The buying price of the components (per ton) are 6, 16 and 10. The components can be mixed along the lines shown in the network to make two possible products that are required to have sulphur contents below 2.5% and 1.5% respectively. Of the 2.5% product a maximum of 1 ton can be sold at a price of 9 per ton. Of the 1.5% product a maximum of 2 tonnes can be sold at a price of 15 per ton. There are a maximum of 3 tons of each component available. The aim is to determine how much of each product should be made to maximise net profit. Note that the pool mixes its inputs, so the sulphur content going out of the pool is the same in all directions. Part of the problem is to determine the proportion of components 1 and 2 in the pool (and hence the sulphur content in the pool).

![](haverly1.png)

Let $q_i : 0 \leq q_i \leq 1$ be the proportion of component $i$ in the pool, $y_k$ the flow (in tons) from the pool to product $k$ and $z_k$ the flow (again in tons) from component 3 to product $k$. Then the problem can be formulated by the following constraints:

\begin{align*}
    q_1 + q_2 &= 1 \\
    y_1(3q_1 + 1q_2) + 2z_1 &\leq 2.5(y_1+z_1) \\
    y_2(3q_1 + 1q_2) + 2z_2 &\leq 1.5(y_2+z_2) \\
    y_1 + z_1 \leq 1 \\
    y_2 + z_2 \leq 2 \\
    q_1(y_1 + y_2) \leq 3 \\
    q_2(y_1 + y_2) \leq 3 \\
    z_1 + z_2 \leq 3
\end{align*}

and objective

$$
    \min\left( -9(y_1 + z_1) -15(y_2 + z_2) + 6q_1(y_1+y_2) + 16q_2(y_1 + y_2) + 10(z_1 + z_2) \right)
$$

with bounds

$$
    q_i, y_i, z_i \geq 0,\quad q_i \leq 1.
$$

### (a)

The above Pooling Problem is coded by the files `NonlinearProblem.py` and `ph1sm.nl`. The Python class `NonlinearProblem` provides methods to evaluate the objective and constraint function and its derivatives (as well as providing information about the problem, such as number of variables and constraints). The header of `NonlinearProblem.py` will explain more. The model itself is given in the nl-file `ph1sm.nl`. This contains a (almost human readable) description of the evaluation trees of the functions defining the problem. The nl-file format is a (quasi) standard used by many modelling systems to communicate constrained nonlinear optmization problems to solvers. Finally `L2PenaltyClass.py` transforms the nonlinear constrained problem into an unconstrained problem by using the $L_2$-penalty function. This can then be solved with a line search method as in previous lab sessions through `CallLineSearch.py`.

Have a look at the corresponding section (`l2pen`) of `CallLineSearch.py` to see how everything is set up. In particular note the line `l2penClass.set_fact(5.0)` that sets the penalty parameter $c$ to $c = 5$. By default the line search is set up to use BFGS with the Wolfe linesearch.

Try running `CallLineSearch.py` and see what happens. _Note: that depending on your setup, python might complain that it does not find the data file `ph1sm.nl`. If that happens you will likely specify the folder in which the file is located in the statement `l2penClass = L2PC("ph1sm.nl")` in `CallLineSearch.py.`_

### (b)

We see that the penalty parameter $c$ is not set large enough, resulting in an unbounded problem. Increase the parameter to $c = 30$.

### (c)

This converges quite fast, but to a solution that clearly does not satisfy the constraints (some variables have negative values). Try progressively increasing $c = 300, 3000, 30000$ and observe what happens.

You will notice that the $L_2$-problems become progressively more difficult to solve as $c$ is increased. But for the final value the solution seems to be close enough to feasibility. Try to interpret this solution in terms of the pooling problem (note that $x = (q_1, q_2, y_1, y_2, z_1, z_2)$). **This is part of the assessed task!**

## 2. Pooling problem by SLP

### (a)

Let's leave the $L_2$-penalty approach and instead look at how SLP (Sequential Linear Programming) works for this problem. A rudimentary SLP algorithm is coded in SLP.py. Have a look at the code; by default it really does nothing more than setting up the LP approximation

$$
    \begin{align*}
        \min_x f(x),\ \text{s.t.}\ \underline{c} \leq g(x) &\leq \overline{c} \\
        \underline{x} \leq x &\leq \overline{x}
    \end{align*}
    \qquad\Rightarrow\qquad
    \begin{align*}
        \min_d f(x),\ \text{s.t.}\ \underline{c}-g(x_k) \leq \nabla g(x)^{\top}d &\leq \overline{c}-g(x_k) \\
        \underline{x}-x_k \leq d &\leq \overline{x}-x_k
    \end{align*}
$$

solving this for $d, then update $x_{k+1} = x_k + d$ and iterate (until the predicted step $d$ is small). Try this by running `SLP.py`.

### (b)

You will find that this stops after 50 iterations (the default maximum). Close inspection reveals that the iterations alternate between points with $f = 1.526057$ and $f = 1.000000$. _Note: you might get a different behaviour if your version of SciPy (which include the linear programming
solver) differs from mine._

As an ad-hoc fix, we may want to impose a small trust region (with fixed size) to the direction finding subproblem. Let's try $\rho = 1$. (Change the parameter at the top of `SLP.py`). Surprisingly this actually converges, to the same solution that we have found with the $L_2$-penalty approach, and does so very fast.

## 3. SLP for small example problem.

To get some more understanding we try a smaller example. 

### (a)

For the problem $\min −x$ subject to $y \geq x^2$, $x^2 + y^2 \leq 1$ the solution is at the rightmost intersection of the two constraints, which is $(x, y) = \left( \sqrt{\frac{\sqrt{5}-1}{2}}, \frac{\sqrt{5}-2}{\sqrt{2}} \right) \approx (0.7862, 0.61803)$.

![](ex915.png)

### (b)

This problem can be chosen in SLP.py by setting `problem='ex915'` in the parameter section at the top (it is Example 9.1.5 in Fletcher’s book – hence the name). Try the SLP solver on it. Again it will converge very quickly (this time even without the help of a tight trust region).

### (c)

Indeed SLP usually converges if solution is at a vertex of the feasible region (and the starting point is close enough to that vertex): in that case SLP just has to identify active constraints and after that it is defaulting to Newton’s method for equation solving.

### (d)

Now change the objective function to $\min −x−y$ (which can be done by choosing `problem = 'ex915mod'` in `SLP.py`. In that case the solution is at $(x, y) = \frac{1}{2}(\sqrt{2}, \sqrt{2})$: at this point only the circle constraint $x^2 + y^2 \leq 1$ is active. Now the curvature of the constraints is important. Again try SLP on this. We see that the naive implementation does not converge (and imposing a default trust region of $\rho = 1$ or so does not help either).

### (e)

Indeed, in order to get convergence in this case we need to implement a proper trust region logic. A basic implementation (that uses a merit function with $\gamma = 1$) can be switched on by setting `do_TR_logic = True` at the top of `SLP.py`. Inspect the Python code to see the consequence of this. Try running `SLP.py` again and see what happens. **The remainder of the assessed task follows from here.**

## 4. The NEOS server (if time).

There are a few commercial and academic solvers for nonlinear optimization, but the choice is not nearly as wide and mature as for linear programming. Many of the available algorithms are research codes and take some effort to get working (also depending on your particular operating system). There is, however, a very useful resource that allows you to try solvers (or simply solve a nonlinear optimization problem) online. This is the NEOS server (at https://neos-server.org/neos/, or just google "neos optimization"). NEOS allows you to submit a problem (written in one of the standard modelling systems AMPL or GAMS), queues it for solution and emails you the results. In this task we will submit a problem expressed in the AMPL language (https://ampl.com/) to NEOS.

### (a)

Take the files `SmallNLP.mod` and `filt.opt` from the Lab download for this week. The mod file states the problem and the opt file is a list of instructions of what to do with the problem. While you likely do not know AMPL, both files should be more or less self-explaining. Have a look at them. Go to the NEOS server https://neos-server.org/neos/ to solve the problem with FilterSQP:

- Select "Submit a job to NEOS"
- Choose "filter [AMPL Input]" under the “Nonlinearly constrained optimization” heading.
- Upload SmallNLP.mod as the Model File and filt.opt as the Commands File (leave the Data File entry blank). Then check the "Short Priority" box, enter an email and press “Submit to NEOS"
- You may have to wait a bit, but this should not be more than a couple of minutes. Results will be emailed and also displayed in the browser window (if you left it open).

### (b)

Modify the two files `SmallNLP.mod` and `filt.opt` in order to solve the problem

$$
\begin{gather*}
  \min_x \exp(x_1x_2x_3x_4x_5)\ \text{subject to} \\
  x^2_1 + x^2_2 + x^2_3 + x^2_4 + x^2_5 = 10 \\
  x_2x_3 = 5x_4x_5 \\
  x^3_1 + x^3_2 = −1
\end{gather*}
$$

and submit it to NEOS

### (c)

You will likely find that the problem is reported as locally infeasible. AMPL allows you to define a starting point for a variable by using the form

```ampl
var x1 default 2;
```

instead of

```ampl
var x1;
```

Use this to start the solver from $(−2, 2, 2, −1, −1)^{\top}$. _Note: the *.mod and *.opt files for parts (b) and (c) are included in the Lab download as `Fletcherp327.mod` and `filt_Fletch.opt`).

## 5. Assessed Task.

> (a) Describe the solution to the Haverly-1 pooling problem: what is the proportion of components in the pool and how much of which product should be produced?
>
> (b) In part 3(e) what happens to the trust region size $\rho$ as you approach the solution? Is this what you would expect? If we did use SQP rather than SLP would you expect the same behaviour?

Submission is on the NLO Learn pages (under Assessments) by **Friday 28 March 10am**.
